In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import json
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from pyecharts import options as opts 
from pyecharts.charts import *
from pyecharts.globals import ChartType, SymbolType,ThemeType
from pyecharts.components.table import Table

In [2]:
url="https://api.inews.qq.com/newsqa/v1/query/inner/publish/modules/list?modules=nowConfirmStatis,provinceCompare"
r=requests.get(url)
dic=json.loads(r.text)
lst=[]
for i in dic['data']['provinceCompare'].keys():
    lst.append(i)
count=0
datadic={}
for i in lst:
    url=f"https://api.inews.qq.com/newsqa/v1/query/pubished/daily/list?province={i}"
    datadic[i]={}
    datadic[i]['日期']=[]
    datadic[i]['累计确诊']=[]
    r=requests.get(url)
    dic=json.loads(r.text)
    for j in dic['data']:
        datadic[i]['日期'].append(str(j['year'])+'.'+j['date'])
        datadic[i]['累计确诊'].append(j['confirm'])
lst1=pd.date_range('20200120',datetime.datetime.now().strftime("%Y%m%d")).strftime("%Y.%m.%d").tolist()
df=pd.DataFrame()
df['日期']=lst1
for i in datadic.keys():
    dfc=pd.DataFrame(datadic[i])
    df=pd.merge(df,dfc,on='日期',how='left')
    df.rename(columns={'累计确诊':f'{i}'},inplace=True)
df.fillna(method='bfill',axis=0,inplace=True)
df=df.set_index('日期')

In [3]:
df

,甘肃,天津,海南,福建,江苏,河南,上海,湖南,河北,西藏,...,湖北,澳门,吉林,山西,重庆,云南,山东,四川,内蒙古,北京
日期,,,,,,,,,,,,,,,,,,,,,
2020.01.20,7.0,14.0,11.0,82.0,1.0,1.0,203.0,24.0,65.0,0.0,...,270,0.0,8.0,6.0,6.0,70.0,121.0,90.0,11.0,102.0
2020.01.21,7.0,14.0,11.0,82.0,1.0,1.0,203.0,24.0,65.0,0.0,...,375,0.0,8.0,6.0,6.0,70.0,121.0,90.0,11.0,102.0
2020.01.22,7.0,14.0,11.0,82.0,1.0,5.0,203.0,24.0,65.0,0.0,...,444,1.0,8.0,6.0,9.0,70.0,121.0,90.0,11.0,102.0
2020.01.23,7.0,14.0,11.0,82.0,9.0,9.0,203.0,24.0,65.0,0.0,...,549,2.0,8.0,6.0,27.0,70.0,121.0,90.0,11.0,102.0
2020.01.23,7.0,14.0,11.0,82.0,9.0,9.0,203.0,24.0,65.0,0.0,...,549,2.0,8.0,6.0,27.0,70.0,121.0,90.0,11.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022.05.24,681.0,1940.0,288.0,3201.0,2224.0,3165.0,62816.0,1393.0,2004.0,1.0,...,68398,82.0,40287.0,420.0,709.0,2136.0,2735.0,2312.0,1753.0,3250.0
2022.05.25,681.0,1951.0,288.0,3212.0,2225.0,3174.0,62865.0,1393.0,2004.0,1.0,...,68398,82.0,40287.0,420.0,710.0,2140.0,2735.0,2314.0,1753.0,3287.0
2022.05.26,681.0,1958.0,288.0,3220.0,2230.0,3174.0,62911.0,1393.0,2004.0,1.0,...,68398,82.0,40289.0,420.0,711.0,2143.0,2735.0,2320.0,1753.0,3310.0


In [4]:
df.to_excel("china_province.xlsx",sheet_name="中国各省份累计疫情数据")

In [5]:
url1="https://ncov.dxy.cn/ncovh5/view/pneumonia"
head={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"}
r=requests.get(url1,headers=head)
r.encoding=r.apparent_encoding
text=r.text
soup=BeautifulSoup(text)
datatotal=soup.select('body #getListByCountryTypeService2true')[0].text
datatotaljson=json.loads(re.findall(r'window.getListByCountryTypeService2.*?.=.(.*)\}catch',datatotal,re.S)[0])
for p in range(len(datatotaljson)):
    if datatotaljson[p]['countryFullName'] == 'China':
        location=p    
chinaurl=datatotaljson[location]['statisticsData']
r=requests.get(chinaurl)
datachinajson=json.loads(r.text)
with open("chinatotal.json","w",encoding="utf-8") as f:
    json.dump(datachinajson,f,indent=4)
chinadata=datachinajson['data']
columns=['累计确诊','累计确诊新增','累计治愈','累计治愈新增','现存确诊','现存确诊新增','日期','累计死亡','累计死亡新增','高风险地区数','中风险地区数','境外输入','境外输入新增']
df1=pd.DataFrame(chinadata)
df1.columns=columns
df1=df1.set_index('日期')
df1.drop(columns=['高风险地区数','中风险地区数'],inplace=True)

In [6]:
df1

,累计确诊,累计确诊新增,累计治愈,累计治愈新增,现存确诊,现存确诊新增,累计死亡,累计死亡新增,境外输入,境外输入新增
日期,,,,,,,,,,
20200119,217,217,25,25,188,188,4,4,0,0
20200120,291,74,25,0,260,72,6,2,0,0
20200121,440,149,25,0,406,146,9,3,0,0
20200122,571,131,28,3,526,120,17,8,0,0
20200123,835,264,34,6,776,250,25,8,0,0
...,...,...,...,...,...,...,...,...,...,...
20220523,1942240,66461,290057,432,1636153,65988,16030,41,18521,15
20220524,2024766,82526,290494,437,1718200,82047,16072,42,18536,15
20220525,2114305,89539,291008,514,1807146,88946,16151,79,18562,26


In [7]:
df1.to_excel("chinatotal.xlsx",index=True)

In [10]:
line1 = Line(init_opts=opts.InitOpts(bg_color='rgb(64, 224, 208)')) 
line1.add_xaxis(df1.index.to_list())
line1.add_yaxis('新增确诊', df1['累计确诊新增'].tolist(), yaxis_index=1, color='#E3170D', is_smooth=True, symbol="none")
line1.add_yaxis('累计确诊', df1['累计确诊'].tolist(), yaxis_index=0, color='#FFFF00', is_smooth=True, symbol="none")
line1.add_yaxis('累计治愈', df1['累计治愈'].tolist(), yaxis_index=0, color='#00FF00', is_smooth=True, symbol="none")
line1.add_yaxis('累计死亡', df1['累计死亡'].tolist(), yaxis_index=1, color='#F0FFFF', is_smooth=True, symbol="none")
line1.add_yaxis('现存确诊', df1['现存确诊'].tolist(), yaxis_index=1, color='#8A2BE2', is_smooth=True, symbol="none")
line1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
line1.extend_axis(yaxis=opts.AxisOpts( axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))))
line1.set_global_opts(title_opts=opts.TitleOpts(title='中国疫情变化', pos_left = 'right', padding=[1,5], title_textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     xaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))),
                     yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#FFFF99'))),
                     legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", textstyle_opts=opts.TextStyleOpts(color='#FFFF99')),
                     datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
                    ) 

line1.render_notebook()